In [360]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [361]:
df = pd.read_csv('../../data/df_rfm.csv')

df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer_ID,Country,TotalSales
0,489434,85048,15CM_CHRISTMAS_GLASS_BALL_20_LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United_Kingdom,83.40
1,489434,22041,"RECORD_FRAME_7""_SINGLE_SIZE",48,2009-12-01 07:45:00,2.10,13085,United_Kingdom,100.80
2,489434,21232,STRAWBERRY_CERAMIC_TRINKET_BOX,24,2009-12-01 07:45:00,1.25,13085,United_Kingdom,30.00
3,489434,22064,PINK_DOUGHNUT_TRINKET_POT,24,2009-12-01 07:45:00,1.65,13085,United_Kingdom,39.60
4,489434,21871,SAVE_THE_PLANET_MUG,24,2009-12-01 07:45:00,1.25,13085,United_Kingdom,30.00
...,...,...,...,...,...,...,...,...,...
713597,581587,22613,PACK_OF_20_SPACEBOY_NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,10.20
713598,581587,22899,CHILDREN'S_APRON_DOLLY_GIRL,6,2011-12-09 12:50:00,2.10,12680,France,12.60
713599,581587,23254,CHILDRENS_CUTLERY_DOLLY_GIRL,4,2011-12-09 12:50:00,4.15,12680,France,16.60
713600,581587,23255,CHILDRENS_CUTLERY_CIRCUS_PARADE,4,2011-12-09 12:50:00,4.15,12680,France,16.60


In [362]:
# prend seulement les dates de 2009 et decembre 2010 separer le data frame en deu
df_2009_2010 = df[(df['InvoiceDate'] >= '2009-12-01') & (df['InvoiceDate'] < '2010-12-01')]
df_2011 = df[df['InvoiceDate'] >= '2010-12-01']

In [363]:
df_2009_2010

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer_ID,Country,TotalSales
0,489434,85048,15CM_CHRISTMAS_GLASS_BALL_20_LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United_Kingdom,83.4
1,489434,22041,"RECORD_FRAME_7""_SINGLE_SIZE",48,2009-12-01 07:45:00,2.10,13085,United_Kingdom,100.8
2,489434,21232,STRAWBERRY_CERAMIC_TRINKET_BOX,24,2009-12-01 07:45:00,1.25,13085,United_Kingdom,30.0
3,489434,22064,PINK_DOUGHNUT_TRINKET_POT,24,2009-12-01 07:45:00,1.65,13085,United_Kingdom,39.6
4,489434,21871,SAVE_THE_PLANET_MUG,24,2009-12-01 07:45:00,1.25,13085,United_Kingdom,30.0
...,...,...,...,...,...,...,...,...,...
337794,536364,21175,GIN_+_TONIC_DIET_METAL_SIGN,12,2010-11-30 19:35:00,2.10,14441,United_Kingdom,25.2
337795,536364,21524,DOORMAT_SPOTTY_HOME_SWEET_HOME,2,2010-11-30 19:35:00,7.95,14441,United_Kingdom,15.9
337796,536364,21098,CHRISTMAS_TOILET_ROLL,12,2010-11-30 19:35:00,1.25,14441,United_Kingdom,15.0
337797,536364,21181,PLEASE_ONE_PERSON_METAL_SIGN,24,2010-11-30 19:35:00,2.10,14441,United_Kingdom,50.4


In [364]:
df_2011

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer_ID,Country,TotalSales
337799,536365,71053,WHITE_METAL_LANTERN,6,2010-12-01 08:26:00,3.39,17850,United_Kingdom,20.34
337800,536365,22752,SET_7_BABUSHKA_NESTING_BOXES,2,2010-12-01 08:26:00,7.65,17850,United_Kingdom,15.30
337801,536365,21730,GLASS_STAR_FROSTED_T-LIGHT_HOLDER,6,2010-12-01 08:26:00,4.25,17850,United_Kingdom,25.50
337802,536366,22633,HAND_WARMER_UNION_JACK,6,2010-12-01 08:28:00,1.85,17850,United_Kingdom,11.10
337803,536366,22632,HAND_WARMER_RED_POLKA_DOT,6,2010-12-01 08:28:00,1.85,17850,United_Kingdom,11.10
...,...,...,...,...,...,...,...,...,...
713597,581587,22613,PACK_OF_20_SPACEBOY_NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,10.20
713598,581587,22899,CHILDREN'S_APRON_DOLLY_GIRL,6,2011-12-09 12:50:00,2.10,12680,France,12.60
713599,581587,23254,CHILDRENS_CUTLERY_DOLLY_GIRL,4,2011-12-09 12:50:00,4.15,12680,France,16.60
713600,581587,23255,CHILDRENS_CUTLERY_CIRCUS_PARADE,4,2011-12-09 12:50:00,4.15,12680,France,16.60


In [365]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import pandas as pd
from sklearn.preprocessing import StandardScaler

# S'assurer que InvoiceDate est au format datetime
df_2009_2010['InvoiceDate'] = pd.to_datetime(df_2009_2010['InvoiceDate'])
df_2011['InvoiceDate'] = pd.to_datetime(df_2011['InvoiceDate'])

# Définir la liste des décalages temporels
time_offsets = [
    pd.DateOffset(days=1),
    pd.DateOffset(days=15),
    pd.DateOffset(months=1),
    pd.DateOffset(months=2),
    pd.DateOffset(months=3),
    pd.DateOffset(months=4),
    pd.DateOffset(months=5),
    pd.DateOffset(months=6),
    pd.DateOffset(months=7),
    pd.DateOffset(months=8),
]

# Initialiser une liste pour stocker les valeurs de ARI
ari_values = []

# Définir les labels initiaux comme None
labels_prev = None

# Pour chaque décalage temporel...
for offset in time_offsets:
    # Ajouter les données du décalage à la dataframe 2009-2010
    offset_date = df_2011['InvoiceDate'].min() + offset
    df_2009_2010_extended = pd.concat([df_2009_2010, df_2011[df_2011['InvoiceDate'] <= offset_date]])

    # Calculer les valeurs RFM
    rfm = df_2009_2010_extended.groupby(['Customer_ID']).agg({
        'InvoiceDate': lambda x: (df_2009_2010_extended['InvoiceDate'].max() - x.max()).days,
        'Invoice': 'nunique',
        'TotalSales': 'sum'})

    # Normaliser les valeurs RFM
    scaler = StandardScaler()
    rfm_normalized = scaler.fit_transform(rfm)

    # Appliquer le clustering K-Means pour segmenter les clients
    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
    kmeans.fit(rfm_normalized)
    rfm['Cluster'] = kmeans.labels_

    # Si ce n'est pas la première itération, calculer l'indice de Rand ajusté
    if labels_prev is not None:
        # Garder seulement les clients communs dans les deux itérations
        common_customers = set(rfm.index).intersection(set(labels_prev.index))
        labels_prev_common = labels_prev[labels_prev.index.isin(common_customers)]
        labels_common = rfm['Cluster'][rfm.index.isin(common_customers)]

        # Calculer l'indice de Rand ajusté
        ari = adjusted_rand_score(labels_prev_common, labels_common)
        ari_values.append(ari)

    # Mettre à jour les labels précédents pour la prochaine itération
    if labels_prev is None:
        labels_prev = rfm['Cluster'].copy()

# Créer un graphique en ligne des valeurs de ARI au fil du temps
time_offsets_str = ['1 Jour', '15 Jours', '1 Mois', '2 Mois', '3 Mois', '4 Mois', '5 Mois', '6 Mois', '7 Mois', '8 Mois']
ari_values.insert(0, 1)  # Ajouter 1 pour l'itération initiale
plt.figure(figsize=(10, 6))
plt.plot(time_offsets_str, ari_values, '-o')
plt.title('Indice de Rand Ajusté au fil du temps')
plt.xlabel('Décalage temporel')
plt.ylabel('Indice de Rand Ajusté')
plt.show()


/tmp/ipykernel_9290/1863173212.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2009_2010['InvoiceDate'] = pd.to_datetime(df_2009_2010['InvoiceDate'])
/tmp/ipykernel_9290/1863173212.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2011['InvoiceDate'] = pd.to_datetime(df_2011['InvoiceDate'])


KeyboardInterrupt: 

In [ ]:
# met les ari value dans un tableau avec les dates
ari_values_df = pd.DataFrame(ari_values, columns=['ari_values'])

ari_values_df['time_offsets_str'] = time_offsets_str

ari_values_df

,ari_values,time_offsets_str
0,1.000000,1 Jour
1,0.933054,15 Jours
2,0.918347,1 Mois
3,0.829287,2 Mois
4,0.729906,3 Mois
5,0.596747,4 Mois
6,0.269424,5 Mois
7,0.043280,6 Mois
8,0.049671,7 Mois
9,0.055974,8 Mois


In [ ]:
common_customers

{12346,
 12347,
 12348,
 12349,
 12350,
 12352,
 12353,
 12354,
 12355,
 12356,
 12357,
 12358,
 12359,
 12360,
 12361,
 12362,
 12365,
 12366,
 12368,
 12369,
 12370,
 12371,
 12372,
 12373,
 12374,
 12376,
 12377,
 12378,
 12379,
 12380,
 12383,
 12385,
 12386,
 12387,
 12388,
 12389,
 12391,
 12392,
 12393,
 12394,
 12395,
 12396,
 12399,
 12400,
 12401,
 12402,
 12406,
 12407,
 12408,
 12409,
 12410,
 12411,
 12412,
 12413,
 12414,
 12415,
 12416,
 12417,
 12418,
 12419,
 12421,
 12422,
 12423,
 12424,
 12425,
 12426,
 12427,
 12428,
 12429,
 12431,
 12432,
 12433,
 12434,
 12435,
 12437,
 12439,
 12440,
 12441,
 12443,
 12444,
 12445,
 12447,
 12449,
 12450,
 12451,
 12454,
 12455,
 12456,
 12457,
 12458,
 12459,
 12460,
 12462,
 12463,
 12464,
 12465,
 12467,
 12468,
 12469,
 12471,
 12472,
 12473,
 12474,
 12475,
 12476,
 12477,
 12480,
 12481,
 12482,
 12483,
 12484,
 12485,
 12486,
 12487,
 12489,
 12490,
 12492,
 12493,
 12494,
 12495,
 12496,
 12497,
 12499,
 12500,
 12501,
